In [129]:
from torchvision.datasets import SUN397
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import torch
import torch.nn as nn
import tqdm
import os
from IPython.display import clear_output
import numpy as np

import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import MultiStepLR
import torchvision.models as models
import torch.nn.utils.prune as prune
import matplotlib.pyplot as plt

In [130]:
from sklearn.model_selection import train_test_split

In [131]:
#!g1.1
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


## Обучение

In [132]:
%pip install wandb -qU
import wandb

In [133]:
class SUN397Dataset(Dataset):
    def __init__(self, root, train=True, download=True):
        transform = T.Compose([
          T.Resize(256),
          T.RandomHorizontalFlip(),
          T.ToTensor(),
          T.Normalize(mean=[0.485, 0.456, 0.486], std=[0.229 , 0.224, 0.225]),
        ]) 
        self.tv_dataset = SUN397(
            root,
            download=download,
            transform=transform,
        )

    def __len__(self):
        return len(self.tv_dataset)

    def __getitem__(self, ix):
        return self.tv_dataset[ix]


In [134]:
def prune_net(net, prune_ratio=0.3, method="l1"):
    parameters_to_prune = [
        (module, "weight") for module in filter(lambda m: type(m) == torch.nn.Conv2d, net.modules())
    ]
    
    prune.global_unstructured(
        parameters_to_prune,
        pruning_method=prune.L1Unstructured,
        amount=prune_ratio,
    )

def copy_weights_layer(layer_unpruned, layer_pruned):
    with torch.no_grad():
      if "weight" in layer_pruned.state_dict():
        layer_pruned.weight.copy_(layer_unpruned.weight)
      if "weight_orig" in layer_pruned.state_dict():
        layer_pruned.weight_orig.copy_(layer_unpruned.weight)
      if "bias" in layer_pruned.state_dict():
        layer_pruned.bias.copy_(layer_unpruned.bias)
      if "running_mean" in layer_pruned.state_dict():
        layer_pruned.running_mean.copy_(layer_unpruned.running_mean)
      if "running_var" in layer_pruned.state_dict():
        layer_pruned.running_var.copy_(layer_unpruned.running_var)

def copy_weights_net(net_unpruned, net_pruned):
    zipped = zip(net_unpruned.modules(), net_pruned.modules())

    for layer_unpruned, layer_pruned in zipped:
      if "weight" in layer_unpruned.state_dict():
        copy_weights_layer(layer_unpruned, layer_pruned)


In [135]:
#!g1.1
# kwargs = dict(
#     num_classes=397,
# )

# base_weights_net = models.resnet18(**kwargs).to(device)
# torch.save(base_weights_net, "base_weights_net")

In [136]:
# os.mkdir("resnet18+SUN397_models")

In [137]:
#!g1.1
def loop_dataloader(dataloader):
    while True:
        for x in iter(dataloader):
            yield x


def train(model, dataloader_train, loss_inst, optimizer, max_iter=10_000,
          dataloader_val=None, val_freq=500, scheduler=None):
    global device
    iterable = loop_dataloader(dataloader_train)
    iterable = tqdm.tqdm(iterable, total=max_iter)
    it = 0
    for X_batch, y_batch in iterable:
        if it == max_iter:
            break
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        logit_batch = model(X_batch)

        loss = loss_inst(logit_batch, y_batch)
        if dataloader_val is not None:
            wandb.log({"loss": loss}, step=it)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if scheduler is not None:
          scheduler.step()

        if it % val_freq == 0 and dataloader_val is not None:
            model.eval()
            is_equal = []

            for X_batch_val, y_batch_val in dataloader_val:
                X_batch_val = X_batch_val.to(device)
                y_batch_val = y_batch_val.to(device)
                is_equal.append(
                    model(X_batch_val).argmax(dim=-1) == y_batch_val
                )

            is_equal_t = torch.cat(is_equal)
            acc = is_equal_t.sum() / len(is_equal_t)
            wandb.log({"accuracy_val": acc}, step=it)
            model.train()

        it += 1
        
def experiment(MAX_ITER, PRUNE_ITER, PRUNE_RATIO , RANDOM_STATE, SCHEDULE, LR, project_name):
    args = {"max-iter" : 30000, "batch-size" : 64, "prune-iter" : 1, "prune-ratio" : 0.2, "prune-method" : "l1", "val-freq" : 250, "random-state" : 1}
    
    
    os.mkdir("resnet18+SUN397_models/pr" + str(PRUNE_RATIO) + "_sched" + str(int(SCHEDULE)) + "_lr" + str(LR) + "_iters" + str(MAX_ITER))
    

    args["max-iter"] = MAX_ITER
    args["prune-iter"] = PRUNE_ITER
    args["prune-ratio"] = PRUNE_RATIO
    args["random-state"] = RANDOM_STATE
    
    wandb.init(
        project=project_name,
        entity="bspanfilov",
        force=True,
        name="pr" + str(PRUNE_RATIO) + "_sched" + str(int(SCHEDULE)) + "_lr" + str(LR),
        config=args,
    )
    wandb.define_metric("accuracy_val", summary="max")
    
    dataset = SUN397Dataset("data2", download=True)

    dataset_val, dataset_train = train_test_split(dataset,
                                                  test_size=0.2, train_size=0.8, 
                                                  random_state=42, shuffle=True
    )

    if args["random-state"] is not None:
        torch.manual_seed(args["random-state"])

    dataloader_train = DataLoader(
        dataset_train, batch_size=args["batch-size"], shuffle=True
    )
    dataloader_val = DataLoader(
        dataset_val, batch_size=args["batch-size"], shuffle=True
    )

    kwargs = dict(
        num_classes=397,
    )

    net = torch.load("base_weights_net").to(device)
    net_copy = models.resnet18(**kwargs).to(device)
    net_copy.load_state_dict(net.state_dict())


    loss_inst = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(net.parameters(), lr=LR, momentum=0.9)
    if SCHEDULE:
      scheduler = MultiStepLR(optimizer, milestones=[6000,8000], gamma=0.1)
    else:
      scheduler = None

#     per_round_prune_ratio = 1 - (1 - args["prune-ratio"]) ** (1 / args["prune-iter"])
#     prune_net(net, per_round_prune_ratio)
#     copy_weights_net(net_copy, net)

    if args["prune-ratio"] > 0:
        per_round_prune_ratio = 1 - (1 - args["prune-ratio"]) ** (1 / args["prune-iter"])

        per_round_max_iter = int(args["max-iter"])

        for prune_it in range(args["prune-iter"]):
            train(
                net,
                dataloader_train,
                loss_inst,
                optimizer,
                max_iter=per_round_max_iter,
            )

            torch.save(net, "resnet18+SUN397_models/pr" + str(PRUNE_RATIO) + "_sched" + str(int(SCHEDULE)) + "_lr" + str(LR) + "_iters" + str(MAX_ITER) + "/prune_it" + str(prune_it))

            prune_net(net, per_round_prune_ratio)
            copy_weights_net(net_copy, net)

    # Run actual training with a final pruned network
    train(
        net,
        dataloader_train,
        loss_inst,
        optimizer,
        max_iter=args["max-iter"],
        dataloader_val=dataloader_val,
        val_freq=args["val-freq"],
        scheduler=scheduler,
    )

    torch.save(net, "resnet18+SUN397_models/pr" + str(PRUNE_RATIO) + "_sched" + str(int(SCHEDULE)) + "_lr" + str(LR) + "_iters" + str(MAX_ITER) + "/pruned")
    
    wandb.finish()



In [138]:
#!g1.1
# тут просто настройки, с которыми я как раз запускал эксперименты
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
MAX_ITERS=[30000]
PRUNE_ITERS=[5]
PRUNE_RATIOS=[0]
RANDOM_STATES=[2] 
SCHEDULE = [False]
LR = [0.03]
for p in zip(MAX_ITERS, PRUNE_ITERS, PRUNE_RATIOS, RANDOM_STATES, SCHEDULE, LR):
  experiment(p[0], p[1], p[2], p[3], p[4], p[5], "SUN397+resnet18")

cuda:0
Failed download. Trying https -> http instead. Downloading http://vision.princeton.edu/projects/2010/SUN/SUN397.tar.gz to data2/SUN397.tar.gz


 81%|████████  | 31688589312/39077296924 [20:32<04:21, 28241439.60it/s]Exception in thread OutRawRd-stderr:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/home/jupyter/.local/lib/python3.8/site-packages/wandb/sdk/internal/sender.py", line 1229, in _output_raw_reader_thread
    self._output_raw_flush(stream)
  File "/home/jupyter/.local/lib/python3.8/site-packages/wandb/sdk/internal/sender.py", line 1244, in _output_raw_flush
    self._output_raw_file.write(data.encode("utf-8"))
  File "/home/jupyter/.local/lib/python3.8/site-packages/wandb/sdk/lib/filesystem.py", line 79, in write
    super().write(b"\n".join(ret) + b"\n")
  File "/home/jupyter/.local/lib/python3.8/site-packages/wandb/sdk/lib/filesystem.py", line 46, in write
    self.f.flush()
OSError: [Errno 28] No space left on device
Thread Sende

OSError: [Errno 28] No space left on device: 'data2/SUN397.tar.gz'

In [ ]:
#!g1.1
